In [ ]:
## importing relevant libraries
import pandas as pd
import numpy as np
import datetime

In [ ]:
## reading the excel and csv files accordingly, this could be automated using loop if we have more numners of file in folder
data1=pd.read_excel("AUS.xlsx")
data2=pd.read_csv("IND.csv")
data3=pd.read_csv("USA.csv")

In [ ]:
## adding a country column in each file to differentiate them after combining
data1['country']='AUS'
data2['country']='IND'
data3['country']='USA'

In [ ]:
## as seen in this there was a pattern in date , written in reverse, so extracting year from right with four characters,
## similarly done for date and month as well
data3['year']=data3.VaccinationDate.astype(str).str[-4:]
data3['date']=data3.VaccinationDate.astype(str).str[-6:-4]
data3['month']=data3.VaccinationDate.astype(str).str[:-6]
data3.month=data3.month.str.zfill(2)
data3['VaccinationDate_modified']=data3.year+"-"+data3.month+"-"+data3.date
data3['VaccinationDate']=data3.VaccinationDate_modified
data3.drop(columns=['year','date','month','VaccinationDate_modified'],inplace=True)
data3.VaccinationDate=pd.to_datetime(data3.VaccinationDate)
data3

In [ ]:
## converting to date format and excluding incoorect date, at large scale this approcah will not be used , i will have to 
## split month and year and date and check if month lies only between values 1 and 12,
## for dates it would be 1 to 31
## for year it will in for digit but not greater then 2022
## will use to map date and make it or two digit as 2 will not work so will make it 02 , like did in data3
data1['A']=pd.to_datetime(data1['Date of Vaccination'],errors='coerce')
data1['invalid_date']=np.where(data1.A.isna(),data1['Date of Vaccination'],'')
data1['Date of Vaccination']=data1['A']
data1.drop(columns=['A'],inplace=True)
data1

In [ ]:
## combining data to singlke file by making a few changes in column name
data2.VaccinationDate=pd.to_datetime(data2.VaccinationDate)
data2.DOB=pd.to_datetime(data2.DOB)
data=data1.merge(data2.rename(columns={'ID':'Unique ID','Name':'Patient Name','DOB':'Date of Birth'
                                       ,'VaccinationType':'Vaccine Type','VaccinationDate':'Date of Vaccination'}),how='outer')
data=data.merge(data3.rename(columns={'ID':'Unique ID','Name':'Patient Name',
                                     'VaccinationType':'Vaccine Type','VaccinationDate':'Date of Vaccination'}),how='outer')
data

### Mectric 1
CountryName, VaccinationType, No. of vaccinations

In [ ]:
vaccine_type_country=pd.DataFrame(data.groupby(['country','Vaccine Type']).count().reset_index())
vaccine_type_country=vaccine_type_country[['country','Vaccine Type','Unique ID']]
vaccine_type_country

In [ ]:
for index,row in vaccine_type_country.iterrows():
  print("In Country",row[0],"with Vaccine Type",row[1],"the total number of vaccinations given are",row[2])

### Assuming
AUS population being 10, IND population being 30 and USA population being 20 and worldwide population being 100

### Metric 2 
CountryName, % Vaccinated

In [ ]:
percentage_vaccine=data.copy()
percentage_vaccine=pd.DataFrame(percentage_vaccine.groupby(['country']).count().reset_index())
percentage_vaccine=percentage_vaccine[['country','Unique ID']]
percentage_vaccine['percentage_of_people_vaccinated']=np.where(percentage_vaccine.country=='AUS',percentage_vaccine['Unique ID']/10,
                                                              np.where(percentage_vaccine.country=='IND',percentage_vaccine['Unique ID']/30,
                                                                      percentage_vaccine['Unique ID']/20))
percentage_vaccine['percentage_of_people_vaccinated']=round((percentage_vaccine['percentage_of_people_vaccinated']*100),2).astype(str)+"%"
percentage_vaccine.drop(columns=['Unique ID'],inplace=True)
percentage_vaccine

In [ ]:
for index,row in percentage_vaccine.iterrows():
  print("In Country",row[0],"percentage of vaccinated people are",row[1])

### Metric 3
CountryName, % Contribution

In [ ]:
world_level_vaccine=data.copy()
world_level_vaccine=pd.DataFrame(world_level_vaccine.groupby(['country']).count().reset_index())
world_level_vaccine=world_level_vaccine[['country','Unique ID']]
world_level_vaccine['world_level_percentage']=(round(world_level_vaccine['Unique ID']/100,2)*100).astype(str)+"%"
world_level_vaccine=world_level_vaccine[['country','world_level_percentage']]
world_level_vaccine

In [ ]:
for index,row in world_level_vaccine.iterrows():
  print("In Country",row[0],"percentage of vaccinated people with respect to population of world",row[1])

In [ ]:
total_among_all_three_countries_people_vaccinated=((data['Unique ID'].count()/100)*100).astype(str)+"%"
print('total number of people vaccinated with respect to world among all these three countries are : ',total_among_all_three_countries_people_vaccinated)